***Group 21***
- Jiaming Chang
- Qianluo Zhu

In [1]:
#import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from scipy.stats import poisson
from scipy.stats import norm
%matplotlib inline

print("packages imported")

packages imported


## **Problem Statement**:  

Develop a model of the charging cost, when we have information about battery capacity, energy consumed, charging duration, charging rate, and distance driven

## **Variables and parameters**:  


## **Assumptions and Constraints:**  


## **Build Solutions:**  

## **Analyze and Assess:**

## **Conclusion and Report**
